In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle
import os

In [2]:
EICU_DATA_PATH = '../train/eicu/'
EICU_lab = pd.read_csv(EICU_DATA_PATH + 'lab.csv')
EICU_prescrip = pd.read_csv(EICU_DATA_PATH + 'medication.csv')
EICU_input = pd.read_csv(EICU_DATA_PATH + 'infusionDrug.csv')
EICU_label = pd.read_csv('../train/labels/eicu_labels.csv')

/tmp/ipykernel_96496/1654522396.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  EICU_prescrip = pd.read_csv(EICU_DATA_PATH + 'medication.csv')
/tmp/ipykernel_96496/1654522396.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  EICU_input = pd.read_csv(EICU_DATA_PATH + 'infusionDrug.csv')


### Lab table 중복처리
labmeasurenamesystem, labmeasurenameinterface -> labmeasurename

In [4]:
EICU_lab.isna().sum()

labid                           0
patientunitstayid               0
labresultoffset                 0
labtypeid                       0
labname                         0
labresult                   32375
labresulttext                1135
labmeasurenamesystem       192546
labmeasurenameinterface    242225
labresultrevisedoffset          0
dtype: int64

In [6]:
EICU_lab['labmeasurename'] = np.where(pd.notnull(EICU_lab['labmeasurenamesystem']) == True, 
                          EICU_lab['labmeasurenamesystem'], EICU_lab['labmeasurenameinterface'])

In [7]:
EICU_lab.isna().sum()


labid                           0
patientunitstayid               0
labresultoffset                 0
labtypeid                       0
labname                         0
labresult                   32375
labresulttext                1135
labmeasurenamesystem       192546
labmeasurenameinterface    242225
labresultrevisedoffset          0
labmeasurename             102145
dtype: int64

### Dictionary

In [32]:
def make_data(lab, prescrip, input_):
    
    lab_columns = ['labresultoffset', 'labtypeid', 'labname', 'labresult', 'labresulttext', 'labmeasurename', 'labresultrevisedoffset']
    prescrip_columns = ['drugorderoffset', 'drugstartoffset', 'drugivadmixture', 'drugordercancelled', 'drugname', 'drughiclseqno', 'dosage', 'routeadmin', 'frequency', 'loadingdose', 'prn', 'drugstopoffset', 'gtc']
    input_columns = ['infusionoffset', 'drugname', 'drugrate', 'infusionrate', 'drugamount', 'volumeoffluid', 'patientweight']
    
    data = {}
    labs = []
    prescrips = []
    inputs = []
    
    for i in range(len(lab)):
        by_lab={}
        for col in lab_columns:
            by_lab[col] = lab[col].iloc[i]
        labs.append(by_lab)
        
    for i in range(len(prescrip)):
        by_prescrip={}
        for col in prescrip_columns:
            by_prescrip[col] = prescrip[col].iloc[i]
        prescrips.append(by_prescrip)
        
    for i in range(len(input_)):
        by_input={}
        for col in input_columns:
            by_input[col] = input_[col].iloc[i]
        inputs.append(by_input)

    data['labs'] = labs
    data['prescrips'] = prescrips
    data['inputs'] = inputs

    return data

In [33]:

def make_stayid_data(lab_data, prescrip_data, input_data, label_data):
    final = []
    for id in tqdm(label_data['patientunitstayid'].unique()):
        dict_by_icuid = {}
        
        lab = EICU_lab[lab_data['patientunitstayid'] == id]
        prescrip = EICU_prescrip[prescrip_data['patientunitstayid'] == id]
        input_ = EICU_input[input_data['patientunitstayid'] == id]
    
        data = make_data(lab, prescrip, input_)
        
        dict_by_icuid['patientunitstayid'] = id
        dict_by_icuid['label'] = label_data[label_data['patientunitstayid']==id].labels.iloc[0]
        dict_by_icuid['data'] = data
        
        final.append(dict_by_icuid)
    return final

In [34]:
EICU_final = make_stayid_data(EICU_lab, EICU_prescrip, EICU_input, EICU_label)

100%|██████████| 101851/101851 [11:40<00:00, 145.41it/s]


In [37]:
EICU_final[0]['data'].keys()

dict_keys(['labs', 'prescrips', 'inputs'])

In [38]:
with open('./eicu.pickle', 'wb') as f:
    pickle.dump(EICU_final, f)